# Imports

In [1]:
import os
import pprint
from langchain.llms.openai import OpenAI
import warnings
warnings.filterwarnings("ignore")

os.environ["SERPER_API_KEY"] = config.serper_api["api_key"]
os.environ["OPENAI_API_KEY"] = config.openai_default["api_key"]

### Mock Portfolio - Ticker : Num of Shares

In [2]:
financial_portfolio = {
                        'AAPL': 10,
                        'FFIC': 5,
                        'CHE': 45,
                        'DAL': 13,
                        'TSLA': 4
                      }

# Create our Custom Agent

In [5]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.utilities import GoogleSerperAPIWrapper ##Tool for connecting our LLM to the internet
from SentimentTool import SentimentTool ##A custom tool that our model can interact with

llm= OpenAI(temperature=0)

In [6]:
sentiment = SentimentTool()
tools = [
    Tool(
        name="Sentiment Analysis",
        func = sentiment.run,
        description="useful for when wanting to find a single stock ticker's sentiment. enter one ticker at a time"    
    )
]

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [39]:
sentiment_prompt = f"""Given the portfolio Portfolio:{str(financial_portfolio)} extract the tickers and find each market
sentiment. Return in dictionary format the ticker as the key and it's market sentiment as it's value"""

sentiment_dict = agent.run(sentiment_prompt)
print(sentiment_dict)

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.




> Entering new AgentExecutor chain...
 I need to find the sentiment of each ticker in the portfolio
Action: Sentiment Analysis
Action Input: 'AAPL'
Observation: -0.022340650553150553
Thought: I need to find the sentiment of the next ticker in the portfolio
Action: Sentiment Analysis
Action Input: 'FFIC'
Observation: 0.13273849206349206
Thought: I need to find the sentiment of the next ticker in the portfolio
Action: Sentiment Analysis
Action Input: 'CHE'
Observation: 0.19719247967479678
Thought: I need to find the sentiment of the next ticker in the portfolio
Action: Sentiment Analysis
Action Input: 'DAL'
Observation: 0.03460545900178254
Thought: I need to find the sentiment of the next ticker in the portfolio
Action: Sentiment Analysis
Action Input: 'TSLA'
Observation: 0.12209967579015966
Thought: I now know the final answer
Final Answer: {'AAPL': -0.022340650553150553, 'FFIC': 0.13273849206349206, 'CHE': 0.19719247967479678, 'DAL': 0.03460545900178254, 'TSLA': 0.12209967579015966}


In [ ]:
{
 'AAPL': -0.022340650553150553, 
 'FFIC': 0.13273849206349206, 
 'CHE': 0.19719247967479678, 
 'DAL': 0.03460545900178254, 
 'TSLA': 0.12209967579015966
}

# Find the Ticker With A Negative Market Sentiment - Prompt Example

#### This example is to show our LLM can understand and mine through structured data like a dictionary

In [48]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

template = """Find the ticker in {sentiments} with a negative value associated to it. Match the ticker with 
the ticker in {portfolio}. Return in dictionary format the key as the ticker and the value as the number of shares"""

prompt_template = PromptTemplate(input_variables=["portfolio", "sentiments"], template=template)

negative_chain = LLMChain(llm=llm, prompt=prompt_template)

negative_sentiment = negative_chain.run({
    'sentiments': sentiment_dict,
    'portfolio': str(financial_portfolio)

    })

print(negative_sentiment)



{'AAPL': 10}


# Google Search Agent for Current Market Value and Current Share Price

#### First we need the stock with the highest market sentiment

In [45]:
import ast

sentiment_dict1 = ast.literal_eval(sentiment_dict)

max_val = max(sentiment_dict1.values())
max_ticker = ''
for key, value in sentiment_dict1.items():
    if sentiment_dict1[key] == max_val:
        max_ticker = key
        break
max_ticker

'CHE'

# Adjust Your Portfolio to Redistribute Your Shares

### For this we use a Google Tool and a Custom Math Tool

In [51]:
from langchain import LLMMathChain
search = GoogleSerperAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to search for current information on stock prices"
    )
]

In [53]:
from pydantic import BaseModel, Field

class CalculatorInput(BaseModel):
    question: str = Field()
        

tools.append(
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="useful for when you need to answer questions about math",
        args_schema=CalculatorInput
        # coroutine= ... <- you can specify an async method if desired as well
    )
)
search_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [69]:
prompt = f"""You are an AI fluent in Finance, 

Return the total market value by gathering the tickers and number of shares from {negative_sentiment}, make sure to 
multiply the current market price for each ticker by the number of shares provided. If there is only one do the 
math for just that ticker

If there are multiple find each market value and sum all of them together

Only return the number"""

market_val = search_agent.run(prompt)
print(market_val)



> Entering new AgentExecutor chain...
 I need to find the current market price for each ticker and multiply it by the number of shares provided.
Action: Search
Action Input: Current market price for AAPL
Observation: 172.18 +0.34 (0.20%)
Thought: I need to multiply the current market price by the number of shares provided.
Action: Calculator
Action Input: 172.18 * 10

> Entering new LLMMathChain chain...
172.18 * 10
```text
172.18 * 10
```
...numexpr.evaluate("172.18 * 10")...

Answer: 1721.8000000000002
> Finished chain.

Observation: Answer: 1721.8000000000002
Thought: I now know the final answer.
Final Answer: 1721.8000000000002

> Finished chain.
1721.8000000000002


#### Get the current market price of the stock with the highest sentiment 

In [68]:
price_ticker = search_agent.run(f"""What is the current market price of {max_ticker}
Only return the number""")
print(price_ticker)



> Entering new AgentExecutor chain...
 I need to find the current market price of CHE
Action: Search
Action Input: CHE stock price
Observation: 539.61 -2.06 (0.38%)
Thought: I now know the final answer
Final Answer: 539.61

> Finished chain.
539.61


#### Find the shares you are buying 

In [73]:
shares = float(market_val) / float(price_ticker)

3.190823001797595

# Adjust our portfolio using our LLM

#### Simple Sequential Chain Example to Transfer Get our new Portfolio then Sort by Sector

In [75]:
template = """Given this portfolio Portfolio: {portfolio}, adjust the JSON to remove  the stock or stocks found in this 
dictionary Dictionary: {neg_sentiments}.

Then in the portfolio, find the stock Stock: {ticker} and add the value Value: {shares} to it's value

Return the full portfolio with the changes listed and sort the tickers by financial sector. Return format
should be in JSON and tickers should be sorted by financial sector"""

prompt_template = PromptTemplate(input_variables=["portfolio", "neg_sentiments", "shares", "ticker"], template=template)

updated_portfolio_chain = LLMChain(llm=llm, prompt=prompt_template)

updated_portfolio = updated_portfolio_chain.run({
    'neg_sentiments': negative_sentiment,
    'portfolio': str(financial_portfolio),
    'ticker': max_ticker,
    'shares': shares

    })

print(updated_portfolio)

.

{
  "CHE": 48.190823001797595,
  "DAL": 13,
  "FFIC": 5,
  "TSLA": 4
}


## First Chain

In [80]:
template = """Given this portfolio Portfolio: {portfolio}, adjust the JSON to remove  the stock or stocks found in this 
dictionary Dictionary: {neg_sentiments}.

Then in the portfolio, find the stock Stock: {ticker} and add the value Value: {shares} to it's value

Return the full portfolio with the changes listed and sort the tickers by financial sector. Return format
should be in JSON and tickers should be sorted by financial sector"""

prompt_template = PromptTemplate(input_variables=["portfolio", "neg_sentiments", "shares", "ticker"], template=template)

updated_portfolio_chain = LLMChain(llm=llm, prompt=prompt_template,
                                output_key='updated_portfolio')

## Second Chain

In [81]:
template = """You are an AI assistant fluent in Finance. 

Given the following portfolio of tickers and how many shares you hold for that ticker, sort the investments by their respective financial sector and return it in JSON format

Portfolio: {updated_portfolio}"""

prompt_template = PromptTemplate(input_variables=["updated_portfolio"], template=template)

portfolio_sort_chain = LLMChain(llm=llm, prompt=prompt_template, output_key='sorted_portfolio')

# Simple Sequential Chain -- Take the output of the first chain and send it as an input to the second

In [87]:
from langchain.chains import SequentialChain

combined_chain = SequentialChain(chains=[updated_portfolio_chain, portfolio_sort_chain],
                                input_variables=["neg_sentiments", "portfolio", "ticker", "shares"],
                                output_variables=["sorted_portfolio"],
                                verbose=True)

output = combined_chain({
    'neg_sentiments': negative_sentiment,
    'portfolio': str(financial_portfolio),
    'ticker': max_ticker,
    'shares': shares
    })



> Entering new SequentialChain chain...

> Finished chain.


In [91]:
output

{'neg_sentiments': "\n\n{'AAPL': 10}",
 'portfolio': "{'AAPL': 10, 'FFIC': 5, 'CHE': 45, 'DAL': 13, 'TSLA': 4}",
 'ticker': 'CHE',
 'shares': 3.190823001797595,
 'sorted_portfolio': '\n\nJSON Output:\n{\n  "Industrials": {\n    "CHE": 48.190823001797595\n  },\n  "Transportation": {\n    "DAL": 13\n  },\n  "Financials": {\n    "FFIC": 5\n  },\n  "Consumer Discretionary": {\n    "TSLA": 4\n  }\n}'}

In [90]:
output['sorted_portfolio']

'\n\nJSON Output:\n{\n  "Industrials": {\n    "CHE": 48.190823001797595\n  },\n  "Transportation": {\n    "DAL": 13\n  },\n  "Financials": {\n    "FFIC": 5\n  },\n  "Consumer Discretionary": {\n    "TSLA": 4\n  }\n}'